In [2]:
# 라이브러리를 로드합니다.
from bs4 import BeautifulSoup
import requests
import re
import sys
import pprint
import pandas as pd

In [3]:
# 댓글을 달 빈 리스트를 생성합니다.
total=[]

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
df = pd.read_csv('/content/drive/Shareddrives/2022-01_데이터마이닝이론및응용/0_텀프로젝트/0_텀프로젝트 코드/크롤링/네이버기사URL.csv')

In [5]:
urls = list(df['기사URL'])

In [6]:

# 여러 리스트들을 하나로 묶어 주는 함수입니다.
def flatten(l):
  flatList = []
  for elem in l:
  # if an element of a list is a list
  # iterate over this list and add elements to flatList 
    if type(elem) == list:
      for e in elem:
        flatList.append(e)
    else:
      flatList.append(elem)
  return flatList

In [11]:
rec =[]
unrec = []

In [16]:
# 네이버 뉴스 url을 입력합니다.

for url in urls:

  try:
    List=[]
    oid=url.split("oid=")[1].split("&")[0]
    aid=url.split("aid=")[1]
    page=1    
    header = {
        "User-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.181 Safari/537.36",
        "referer":url,
        
    } 
    while True :
        c_url="https://apis.naver.com/commentBox/cbox/web_neo_list_jsonp.json?ticket=news&templateId=default_society&pool=cbox5&_callback=jQuery1707138182064460843_1523512042464&lang=ko&country=&objectId=news"+oid+"%2C"+aid+"&categoryId=&pageSize=20&indexSize=10&groupId=&listType=OBJECT&pageType=more&page="+str(page)+"&refresh=false&sort=FAVORITE" 
    # 파싱하는 단계입니다.
        r=requests.get(c_url,headers=header)
        cont=BeautifulSoup(r.content,"html.parser")    
        total_comm=str(cont).split('comment":')[1].split(",")[0]
    
        match=re.findall('"contents":([^\*]*),"userIdNo"', str(cont))

    # 댓글을 리스트에 중첩합니다.
        List.append(match)
    # 한번에 댓글이 20개씩 보이기 때문에 한 페이지씩 몽땅 댓글을 긁어 옵니다.
        if int(total_comm) <= ((page) * 20):
            break
        else : 
            page+=1

 
    # 리스트 결과입니다.
    total.append(flatten(List))

  except:
    total.append('')

In [18]:
len(total)

1598

In [19]:
len(urls)

1592

In [20]:
total2 = total[6:]

In [21]:
len(total2)

1592

In [22]:
d = {'url': urls, '댓글': total2 }
df_final = pd.DataFrame(data=d)
df_final

,url,댓글
0,https://news.naver.com/main/read.naver?mode=LS...,"[""뭐 맡겨놓았음? ㅋㅋ 그리고 이런 소리 할려면 인권을 중시하는 민주당 180석에..."
1,https://news.naver.com/main/read.naver?mode=LS...,"[""박대표 장애인 협회 토론해서 개망신 당하길 이준석화이팅. 불법집회에 시민들 불편..."
2,https://news.naver.com/main/read.naver?mode=LS...,"[""근데 장애인들이 이석기 석방은 왜하자는거냐"", ""이준석은 장애인을 비난한게 아니..."
3,https://news.naver.com/main/read.naver?mode=LS...,"[""이준석 대표님 정치 잘한다 대한민국은 세금을 공평하게 국민들한테 분배가 되어야 ..."
4,https://news.naver.com/main/read.naver?mode=LS...,"[""남성혐오 하는 페미들 지지한 이재명이나 까보시지"", ""이준석: 한 '시민단체'의..."
...,...,...
1587,https://news.naver.com/main/read.naver?mode=LS...,"[""장애인은 무슨 권리로 비장애인에게 불편을 끼치고 잠깐 참으라하나. 장애인도 국민..."
1588,https://news.naver.com/main/read.naver?mode=LS...,"[""일끝나고 집가는데 무슨 민폔지 모르겠음 지들 권리를 주장하려면 일반시민들한테는 ..."
1589,https://news.naver.com/main/read.naver?mode=LS...,[]
1590,https://news.naver.com/main/read.naver?mode=LS...,"[""장애인분들은 진짜 나라에서 도와드려야합니다 국회의원들 월급줄여서요!!!!"", ""..."


In [23]:
df_final.to_csv('댓글.csv', index=False, encoding='utf-8-sig')

In [15]:
total

[['"뭐 맡겨놓았음? ㅋㅋ 그리고 이런 소리 할려면 인권을 중시하는 민주당 180석에 이야기 하면 될것이고, 지금 아직 5월도 안왔음 , 문재인 정권이고 , 민주당 180석 이다 거기다 가서 말해라 아직 정권이양도 받지 못한곳에 이야기 하지 말고 ㅋ"',
  '"장애인도 법 지키고, 타인 배려해야 한다고 말한 게 사퇴할 일인가요? 그래서 대부분의 정치인이 모른척 입다물고 있는데, 그건 잘 하는 짓이고? 당대표 가 돌직구 날리는 게 보기에 좀 아니다 싶지만, 그게 무슨 사퇴거린가? 피해호소인 부르짖던 민주당 분들은 사퇴하셨나?"',
  '"하이고 이건 또 무슨 단체? 먼 장애인단체가 이렇게 많아? 오만 단체 다 나와서 떠드네. 한국지체장애인협회? 그 큰단체는 오히려 사과했드만 시민들한테 사과부터 하고 원하는거 얘기해라"',
  '"문정권때는 한 마디도 못하던 놈들이  왜 이리 갑자기 떠들어대냐? 꼭 누가 시킨것처럼 ㅋㅋ ㅋ  180석 이면 뭐든 다해줬을텐데"',
  '"준석이 말이 다 틀린말은 아니다."',
  '"3호선 타고 출퇴근하는 사람입니다. 장애인을 위해 개선 되어야할 부분이 너무나 많지만 그렇다고 아무에게나 피해를 줄수 있는 권리가 생기는 것은 아닙니다. 한번은 참았지만 여러번 반복될때 정말 많이 화났어요. 지하철 시위는 정말 아닙니다."',
  '"이런 면에서 이준석 대표를 좋아합니다. 해줄수 있는건 해주고 당장 해줄 수 없는건 사정 설명과 계획을 말 해주는 사회 분위기나 시스템이 되면되는데 이 전 정권에서는 한 마디 말도 못하다가 시민의 발을 무더기로 묶는 행위는 어떤 경우도 이해가 안되죠.역시 이준석씨가 어떤 때는 너무 앞서가는거 같지만 똑똑하고 핵심을 꽉 잡네요."',
  '"장애자 여러분! 장애자도 모든 자유를 누릴 권리 있습니다  준석이 말이 장애인 입장에서는 당연히 고깝겠지만 만민의 입장에서 볼 때 무엇이 그렇게도 화가 나는 일인지 정확히 말해야 나 같이 무식한 국민이 이해를 하죠 그저 장애인을 위한 대책도 없이 그런다고만 하면 그건 아